# HAP 786 WORKSHOP - REMOVE ALGORITHM BIAS 

## DATA PREPARATION

### DATA IMPORT FROM ALL OF US 

In [1]:
import pandas
import os

# This query represents dataset "Diseases_dataset" for domain "condition" and was generated for All of Us Registered Tier Dataset v7
dataset_64782628_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (4274025)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (38003563) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (4152280)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (436665)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) )
            )) c_occurrence 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id"""

dataset_64782628_condition_df = pandas.read_gbq(
    dataset_64782628_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_64782628_condition_df.head(5)

Downloading:   0%|          | 0/3976892 [00:00<?, ?rows/s]

,person_id,condition_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,condition_start_datetime,condition_end_datetime,condition_type_concept_id,condition_type_concept_name,stop_reason,visit_occurrence_id,visit_occurrence_concept_name,condition_source_value,condition_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,condition_status_source_value,condition_status_concept_id,condition_status_concept_name
0,2179796,43531578,Chronic kidney disease due to type 2 diabetes ...,771000119108,SNOMED,2018-09-01 04:00:00+00:00,NaT,32821,EHR billing record,None,<NA>,None,E11.22,45595797,Type 2 diabetes mellitus with diabetic chronic...,E11.22,ICD10CM,None,<NA>,None
1,1622769,81169,Superficial foreign body of finger without maj...,52877004,SNOMED,1997-01-14 00:00:00+00:00,NaT,44786629,Secondary Condition,None,27000000010091692,None,915.6,44830569,Superficial foreign body (splinter) of finger(...,915.6,ICD9CM,None,<NA>,None
2,1557758,4149904,"Attention deficit hyperactivity disorder, comb...",31177006,SNOMED,2022-01-14 00:00:00+00:00,NaT,44786627,Primary Condition,None,44000000000448720,None,F90.2,45552506,"Attention-deficit hyperactivity disorder, comb...",F90.2,ICD10CM,None,<NA>,None
3,2172387,380111,Acute atopic conjunctivitis,67678004,SNOMED,2009-10-04 00:00:00+00:00,NaT,44786627,Primary Condition,None,27000000007016720,None,372.05,44823068,Acute atopic conjunctivitis,372.05,ICD9CM,None,<NA>,None
4,3152137,4332246,Aneurysm,432119003,SNOMED,2007-09-26 04:00:00+00:00,NaT,32821,EHR billing record,None,<NA>,None,437.3,44826646,"Cerebral aneurysm, nonruptured",437.3,ICD9CM,None,<NA>,None


In [2]:
import pandas
import os

# This query represents dataset "Antidepressants" for domain "drug" and was generated for All of Us Registered Tier Dataset v7
dataset_48252559_drug_sql = """
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr             
                        WHERE
                            concept_id IN (21604686, 21604729, 21604788)             
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))  
                    AND (d_exposure.PERSON_ID IN (SELECT
                        distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (38003563) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (4152280)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (436665)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) )
            )) d_exposure 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_standard_concept 
            ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_type 
            ON d_exposure.drug_type_concept_id = d_type.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_route 
            ON d_exposure.route_concept_id = d_route.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.visit_occurrence` v 
            ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_visit 
            ON v.visit_concept_id = d_visit.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` d_source_concept 
            ON d_exposure.drug_source_concept_id = d_source_concept.concept_id"""

dataset_48252559_drug_df = pandas.read_gbq(
    dataset_48252559_drug_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_48252559_drug_df.head(5)

Downloading:   0%|          | 0/155592 [00:00<?, ?rows/s]

,person_id,drug_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,drug_exposure_start_datetime,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,drug_type_concept_name,...,lot_number,visit_occurrence_id,visit_occurrence_concept_name,drug_source_value,drug_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,route_source_value,dose_unit_source_value
0,3049202,715295,duloxetine 20 MG Delayed Release Oral Capsule ...,596928,RxNorm,2018-12-26 14:08:03+00:00,2018-12-26 14:08:03+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None
1,2210255,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2016-08-09 21:01:16+00:00,2016-08-09 21:01:16+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None
2,2693659,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2018-12-31 13:00:00+00:00,2018-12-31 15:12:58.667000+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None
3,1074964,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,2019-03-01 05:00:00+00:00,NaT,NaT,38000177,Prescription written,...,None,<NA>,None,835566,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,26643006,None
4,1114603,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2016-06-12 14:00:00+00:00,2016-06-12 15:17:12.013000+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None


In [3]:
import pandas
import os

# This query represents dataset "Elderly_Dead" for domain "observation" and was generated for All of Us Registered Tier Dataset v7
dataset_36027734_observation_sql = """
    SELECT
        observation.person_id,
        observation.observation_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.observation` observation 
        WHERE
            (
                observation_concept_id IN (4051267, 4061268, 4062254, 4083743, 4132309, 4177807, 4195755, 4302017, 4306655, 441139, 442605, 443882)
            )  
            AND (
                observation.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
                    WHERE
                        ethnicity_concept_id IN (38003563) ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (4152280)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) 
                    AND cb_search_person.person_id NOT IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (436665)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) )
            )) observation"""

dataset_36027734_observation_df = pandas.read_gbq(
    dataset_36027734_observation_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_36027734_observation_df.head(5)

Downloading:   0%|          | 0/1 [00:00<?, ?rows/s]

,person_id,observation_datetime
0,1963312,2019-03-01 11:50:26+00:00


In [4]:
import pandas
import os

# This query represents dataset "Demographic_dataset" for domain "person" and was generated for All of Us Registered Tier Dataset v7
dataset_83327658_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.person` p 
            WHERE
                ethnicity_concept_id IN (38003563) ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (4152280)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (436665)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_83327658_person_df = pandas.read_gbq(
    dataset_83327658_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_83327658_person_df.head(5)

Downloading:   0%|          | 0/9456 [00:00<?, ?rows/s]

,person_id,gender_concept_id,gender,date_of_birth,race_concept_id,race,ethnicity_concept_id,ethnicity,sex_at_birth_concept_id,sex_at_birth
0,1105145,2000000002,"Not man only, not woman only, prefer not to an...",1995-06-20 00:00:00+00:00,2000000001,Another single population,38003563,Hispanic or Latino,2000000009,"Not male, not female, prefer not to answer, or..."
1,1861024,45880669,Male,1954-12-17 00:00:00+00:00,2000000001,Another single population,38003563,Hispanic or Latino,2000000009,"Not male, not female, prefer not to answer, or..."
2,1064452,45878463,Female,1937-02-28 00:00:00+00:00,2000000001,Another single population,38003563,Hispanic or Latino,2000000009,"Not male, not female, prefer not to answer, or..."
3,2569462,2000000002,"Not man only, not woman only, prefer not to an...",1990-03-19 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,2000000009,"Not male, not female, prefer not to answer, or..."
4,3079515,2000000002,"Not man only, not woman only, prefer not to an...",1979-11-03 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,2000000009,"Not male, not female, prefer not to answer, or..."


### DATA CHECKPOINT 

In [5]:
#Rename Dataframes
diseases_df = dataset_64782628_condition_df #all diseases
antidepressant_df = dataset_48252559_drug_df #all antidepressants
dead_df = dataset_36027734_observation_df #all dead
demographics_df = dataset_83327658_person_df #all participant demographics

In [6]:
import pandas as pd 
import numpy as np
import os

### DATA PREPARATION 


#### RECREATE UNIQUE ANTIDEPRESSANT FILES 

In [7]:
antidepressant_df.head()

,person_id,drug_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,drug_exposure_start_datetime,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,drug_type_concept_name,...,lot_number,visit_occurrence_id,visit_occurrence_concept_name,drug_source_value,drug_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,route_source_value,dose_unit_source_value
0,3049202,715295,duloxetine 20 MG Delayed Release Oral Capsule ...,596928,RxNorm,2018-12-26 14:08:03+00:00,2018-12-26 14:08:03+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None
1,2210255,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2016-08-09 21:01:16+00:00,2016-08-09 21:01:16+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None
2,2693659,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2018-12-31 13:00:00+00:00,2018-12-31 15:12:58.667000+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None
3,1074964,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,2019-03-01 05:00:00+00:00,NaT,NaT,38000177,Prescription written,...,None,<NA>,None,835566,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,26643006,None
4,1114603,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2016-06-12 14:00:00+00:00,2016-06-12 15:17:12.013000+00:00,NaT,32818,EHR administration record,...,None,<NA>,None,None,<NA>,None,None,None,26643006,None


In [8]:
ad_df = antidepressant_df[['standard_concept_code','standard_concept_name']].copy()
ad_df.head()

,standard_concept_code,standard_concept_name
0,596928,duloxetine 20 MG Delayed Release Oral Capsule ...
1,993542,24 HR bupropion hydrochloride 150 MG Extended ...
2,993542,24 HR bupropion hydrochloride 150 MG Extended ...
3,835566,imipramine hydrochloride 25 MG Oral Tablet [To...
4,993542,24 HR bupropion hydrochloride 150 MG Extended ...


In [9]:
unique_ad_df = ad_df.drop_duplicates(subset=['standard_concept_code'], keep='first')
unique_ad_df.head()

,standard_concept_code,standard_concept_name
0,596928,duloxetine 20 MG Delayed Release Oral Capsule ...
1,993542,24 HR bupropion hydrochloride 150 MG Extended ...
3,835566,imipramine hydrochloride 25 MG Oral Tablet [To...
7,856762,amitriptyline hydrochloride 100 MG Oral Tablet
8,1000076,doxepin hydrochloride 50 MG Oral Capsule


In [10]:
unique_ad_df.to_csv('./export/unique_ad_df.csv', index = False)

#### PREPARE AND GROUP ANTIDEPRESSANTS DATAFRAME

In [11]:
#Read and groupthe antidepressant grouping file to create ad_grouping_df 
ad_grouping_df = pd.read_csv("./Data folder/unique_ad_grouped.csv")
ad_grouping_df.head()

,standard_concept_code,standard_concept_name,ad_grouping
0,1000048,doxepin hydrochloride 10 MG Oral Capsule,Doxepin
1,1000052,doxepin hydrochloride 10 MG Oral Capsule [Sine...,Doxepin
2,1000054,doxepin hydrochloride 10 MG/ML Oral Solution,Doxepin
3,1000058,doxepin hydrochloride 100 MG Oral Capsule,Doxepin
4,1000062,doxepin hydrochloride 100 MG Oral Capsule [Sin...,Doxepin


In [12]:
ad_grouping_df['ad_grouping'].unique()

array(['Doxepin', 'Other', 'Fluoxetine', 'Trazodone', 'Citalopram',
       'Nortriptyline', 'Bupropion', 'Paroxetine', 'Desvenlafaxine',
       'Mirtazapine', 'Duloxetine', 'Amitriptyline', 'Sertraline',
       'Venlafaxine', 'Escitalopram'], dtype=object)

In [13]:
ad_grouping_df[ad_grouping_df['standard_concept_code'].astype(str).str.contains('OMOP', na=False)]



,standard_concept_code,standard_concept_name,ad_grouping


In [14]:
ad_grouping_df[ad_grouping_df['standard_concept_code'].astype(str).str.contains('1000048', na=False)]


,standard_concept_code,standard_concept_name,ad_grouping
0,1000048,doxepin hydrochloride 10 MG Oral Capsule,Doxepin


In [15]:
print(ad_grouping_df['standard_concept_code'].unique())


[1000048 1000052 1000054 1000058 1000062 1000064 1000068 1000070 1000074
 1000076 1000081 1000091 1000093 1000095 1000097 1000101  104838  104849
  104850   10737 1086769 1086772 1086776 1086778 1086780 1086784 1086786
 1086790  108886   10898 1098649 1098666 1098670 1098672 1098680 1098710
 1099288 1099290 1099296 1099300 1099304 1099316 1100363 1100384 1190110
 1232585 1232591 1298857 1424836 1430122 1430128 1435480 1435483 1439808
 1439810 1439812 1439829 1439838 1439844 1455099  153503 1551468 1551474
   15996 1607617 1652069 1653470 1738483 1738495 1738503 1738511 1738519
 1738523 1738803 1738804 1738805 1738806 1738807 1738808 1790886 1790890
 1790892 1801289 1874553 1874559  197365  198045  198046  198047  200371
  201694  201696  205535  207349  207350  208149  208161  208848  208849
  208850  208851  208853  209329  209339  209350  209391  211322  211323
  211699  211700  212233  213344  213345  226376  248642 2532163    2556
  261135  261282  261287  283406  283407  283485  2

In [16]:
#Convert standard_concept_code to Int64
antidepressant_df['standard_concept_code'] = antidepressant_df['standard_concept_code'].astype('Int64')

#Left join to group the antidepressants based on standard_concept_code
grouped_ad_df = pd.merge(antidepressant_df, ad_grouping_df[['standard_concept_code','ad_grouping']],
                         on='standard_concept_code',how='left')

grouped_ad_df.head()

,person_id,drug_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,drug_exposure_start_datetime,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,drug_type_concept_name,...,visit_occurrence_id,visit_occurrence_concept_name,drug_source_value,drug_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,route_source_value,dose_unit_source_value,ad_grouping
0,3049202,715295,duloxetine 20 MG Delayed Release Oral Capsule ...,596928,RxNorm,2018-12-26 14:08:03+00:00,2018-12-26 14:08:03+00:00,NaT,32818,EHR administration record,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Duloxetine
1,2210255,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2016-08-09 21:01:16+00:00,2016-08-09 21:01:16+00:00,NaT,32818,EHR administration record,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion
2,2693659,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2018-12-31 13:00:00+00:00,2018-12-31 15:12:58.667000+00:00,NaT,32818,EHR administration record,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion
3,1074964,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,2019-03-01 05:00:00+00:00,NaT,NaT,38000177,Prescription written,...,<NA>,None,835566,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,26643006,None,Other
4,1114603,40221872,24 HR bupropion hydrochloride 150 MG Extended ...,993542,RxNorm,2016-06-12 14:00:00+00:00,2016-06-12 15:17:12.013000+00:00,NaT,32818,EHR administration record,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion


In [17]:
#Number of participants prior to dates processing
len(grouped_ad_df['person_id'].unique().tolist())

6263

In [18]:
#Drop unneeded columns will use AD grouping moving forward
grouped_ad_df = grouped_ad_df.drop(columns=['standard_concept_name', 'standard_concept_code'])
grouped_ad_df.head()

,person_id,drug_concept_id,standard_vocabulary,drug_exposure_start_datetime,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,drug_type_concept_name,stop_reason,refills,...,visit_occurrence_id,visit_occurrence_concept_name,drug_source_value,drug_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,route_source_value,dose_unit_source_value,ad_grouping
0,3049202,715295,RxNorm,2018-12-26 14:08:03+00:00,2018-12-26 14:08:03+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Duloxetine
1,2210255,40221872,RxNorm,2016-08-09 21:01:16+00:00,2016-08-09 21:01:16+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion
2,2693659,40221872,RxNorm,2018-12-31 13:00:00+00:00,2018-12-31 15:12:58.667000+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion
3,1074964,778297,RxNorm,2019-03-01 05:00:00+00:00,NaT,NaT,38000177,Prescription written,None,0,...,<NA>,None,835566,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,26643006,None,Other
4,1114603,40221872,RxNorm,2016-06-12 14:00:00+00:00,2016-06-12 15:17:12.013000+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion


#### FILL OUT DATA FOR ANTIDEPRESSANT DATES
Start date(first 3 trials)
End date(first 3 trials)
dosage days (first 3 trials)
upto 4 weeks is just random non adherence

In [19]:
#number of rows with duplicates
grouped_ad_df.shape[0] 

155592

In [20]:
#Remove duplicate rows
grouped_ad_df = grouped_ad_df.drop_duplicates(
            subset=['person_id','drug_exposure_start_datetime','drug_exposure_end_datetime','days_supply','ad_grouping'])

In [21]:
grouped_ad_df.shape[0] #number of rows without duplicates

133852

In [22]:
grouped_ad_df.head()

,person_id,drug_concept_id,standard_vocabulary,drug_exposure_start_datetime,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,drug_type_concept_name,stop_reason,refills,...,visit_occurrence_id,visit_occurrence_concept_name,drug_source_value,drug_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,route_source_value,dose_unit_source_value,ad_grouping
0,3049202,715295,RxNorm,2018-12-26 14:08:03+00:00,2018-12-26 14:08:03+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Duloxetine
1,2210255,40221872,RxNorm,2016-08-09 21:01:16+00:00,2016-08-09 21:01:16+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion
2,2693659,40221872,RxNorm,2018-12-31 13:00:00+00:00,2018-12-31 15:12:58.667000+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion
3,1074964,778297,RxNorm,2019-03-01 05:00:00+00:00,NaT,NaT,38000177,Prescription written,None,0,...,<NA>,None,835566,778297,imipramine hydrochloride 25 MG Oral Tablet [To...,835566,RxNorm,26643006,None,Other
4,1114603,40221872,RxNorm,2016-06-12 14:00:00+00:00,2016-06-12 15:17:12.013000+00:00,NaT,32818,EHR administration record,None,<NA>,...,<NA>,None,None,<NA>,None,None,None,26643006,None,Bupropion


In [23]:
#import library
from datetime import datetime , timedelta

In [24]:
#Intialize ad_dates_df to store dosage dates and days for each antidepressant group per person
#DEfine a dataframe with desired structure
ad_dates_df = pd.DataFrame(columns=['person_id','ad_grouping','trail_number','earliest_ad_start','earliest_ad_end','dosage_days'])

#Check adattypes for match with required processing
print(ad_dates_df.dtypes)

person_id            object
ad_grouping          object
trail_number         object
earliest_ad_start    object
earliest_ad_end      object
dosage_days          object
dtype: object


In [25]:
#change integers to nullable ints
ad_dates_df['person_id'] = ad_dates_df['person_id'].astype('Int64')
ad_dates_df['trail_number'] = ad_dates_df['trail_number'].astype('Int64')
ad_dates_df['dosage_days'] = ad_dates_df['dosage_days'].astype('Int64')

#change dates to proper dates data types
ad_dates_df['earliest_ad_start'] = pd.to_datetime(ad_dates_df['earliest_ad_start'])
ad_dates_df['earliest_ad_end'] = pd.to_datetime(ad_dates_df['earliest_ad_end'])

#print the dataframe with updates datatypes
print(ad_dates_df.dtypes)

#Retain data type for ad_grouping as object / string 

person_id                     Int64
ad_grouping                  object
trail_number                  Int64
earliest_ad_start    datetime64[ns]
earliest_ad_end      datetime64[ns]
dosage_days                   Int64
dtype: object


In [26]:
ad_dates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   person_id          0 non-null      Int64         
 1   ad_grouping        0 non-null      object        
 2   trail_number       0 non-null      Int64         
 3   earliest_ad_start  0 non-null      datetime64[ns]
 4   earliest_ad_end    0 non-null      datetime64[ns]
 5   dosage_days        0 non-null      Int64         
dtypes: Int64(3), datetime64[ns](2), object(1)
memory usage: 124.0+ bytes


In [27]:
#Define function to calculate end dates and dosage days given the values of the end date and days supplies columns
def calculate_dosage_days(row,trial_end, trial_dosage):
    
    #end date and days supply do not have values
    if (pd.isna(row['drug_exposure_end_datetime']) and 
       (pd.isna(row(['days_supply']) or row['days_supply'] == 0))):
        
        #take start date as end date and increment dosage (one time dose)
        trial_end = row['dru_exposure_start_datetime']
        trail_dosage += 1 
        
        #end date has value but days supply does not
    elif(pd.isna(row['days_supply']) or row['days_supply'] == 0 ):
        
        #take current end date and assume some dosage days as time elapse
        trial_end = row['drug_exposure_end_datetime'] 
        date_diff = row['drug_exposure_end_datetime'] - row['drug_exposure_start_datetime']
        trial_dosage += date_diff.days + 1 
        #end date has no value but days supply has
    elif pd.isna(row['drug_exposure_end_datetime']):
        #increment dosage days by days supply and assume same number of days
        #elapsed from start date to get end date
        trial_end = row['drug_exposure_end_daetime'] + timedelta(days=row['days_supply']) - timedelta(days=1)
        trial_dosage += row['days_supply']
        
        #end date and days supply have values
    else:
        #get current end date and increment dosage days by days supply
        trial_end = row['drug_exposure_end_datetime']
        trial_dosage += row['days_supply']
        
        return trial_end, trial_dosage

In [28]:
# Define a function to calculate dosage days (Assuming this function is defined elsewhere)
def calculate_dosage_days(row, trail_end, trail_dosage):
    # Your logic for calculating dosage days goes here
    # Return updated trail_end and trail_dosage
    return trail_end, trail_dosage  # Replace with actual logic

# Define function to process antidepressant dates for each person
def process_person(person, person_subset):
    ad_dates_list = []
    
    # Loop through every antidepressant the person is taking 
    for ad in person_subset['ad_grouping'].unique():
        # Create a subset for the antidepressant for the person
        person_ad_subset = person_subset[person_subset['ad_grouping'] == ad]
        
        # Get the earliest antidepressant start date
        earliest_ad_start = person_ad_subset['drug_exposure_start_datetime'].min()
        
        # Sort dosage data for the person and antidepressant by start date
        person_ad_subset_sorted = person_ad_subset.sort_values(by='drug_exposure_start_datetime')
        
        # Initialize values for end date and dosage trackers 
        earliest_ad_end = earliest_ad_start
        dosage_days = 0
        
        # Initialize trail trackers; there will be a maximum of 3 trails
        trail, trail_end, trail_dosage = 1, earliest_ad_start, 0 
        
        # Loop sorted dosage data and find values for end date and dosage days
        for index, row in person_ad_subset_sorted.iterrows():
            # If start date is more than 4 weeks from current end date or the last row, process trackers
            if (((row['drug_exposure_start_datetime'] - trail_end) > timedelta(days=28)) or
                (index == person_ad_subset_sorted.index[-1])): 
                
                # Take the current trail end and trail dosage to write on the table
                earliest_ad_end, dosage_days = trail_end, trail_dosage
                
                # Append to the dates being tracked
                ad_dates_list.append({
                    'person_id': person, 
                    'ad_grouping': ad,  # Fixed typo from 'ad_gropuing' to 'ad_grouping'
                    'trail_number': trail, 
                    'earliest_ad_start': earliest_ad_start,
                    'earliest_ad_end': earliest_ad_end, 
                    'dosage_days': dosage_days
                })
                
                # If the last trail or last row, then break the loop
                if (trail > 2) or (index == person_ad_subset_sorted.index[-1]):
                    break
                
                # If not the last trail or last row, initialize the next dosage dates
                else:
                    earliest_ad_start = row['drug_exposure_start_datetime']
                    earliest_ad_end = row['drug_exposure_start_datetime']
                    
                # Update trail trackers to start tracking the next trail
                trail_end = row['drug_exposure_start_datetime']
                trail += 1
                trail_dosage = 0
                
            # Current end date in tracker is greater than the date being processed, then skip
            if (trail_end > row['drug_exposure_start_datetime']) or (pd.isna(row['drug_exposure_start_datetime']) and (trail_end > row['drug_exposure_start_datetime'])):
                continue
            
            # Calculate trail end date and dosage for the row
            trail_end, trail_dosage = calculate_dosage_days(row, trail_end, trail_dosage)
    
    return ad_dates_list

        

In [29]:
# Iterate over unique person_ids in grouped_ad_df
ad_dates_df = pd.DataFrame()  # Initialize the DataFrame to store results
for person in grouped_ad_df['person_id'].unique():
    # Create a subset for the person
    person_subset = grouped_ad_df[grouped_ad_df['person_id'] == person]
    # Get the relevant antidepressant trail dates and dosage days for the person
    ad_dates_list = process_person(person, person_subset)
    # Append to the DataFrame
    ad_dates_df = pd.concat([ad_dates_df, pd.DataFrame(ad_dates_list)], ignore_index=True)

ad_dates_df.head(5)

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0
1,3049202,Duloxetine,2,2019-06-23 20:45:38.964000+00:00,2019-06-23 20:45:38.964000+00:00,0
2,3049202,Duloxetine,3,2020-06-20 12:26:00+00:00,2020-06-20 12:26:00+00:00,0
3,3049202,Venlafaxine,1,2008-03-01 19:59:00.040000+00:00,2008-03-01 19:59:00.040000+00:00,0
4,3049202,Venlafaxine,2,2009-09-17 04:06:47.500000+00:00,2009-09-17 04:06:47.500000+00:00,0


In [30]:
#create the new columns for dose-70p
ad_dates_df['dose_70p'] = np.where(ad_dates_df['dosage_days'] >= 70 , 1,0)

ad_dates_df.head(5)

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0
1,3049202,Duloxetine,2,2019-06-23 20:45:38.964000+00:00,2019-06-23 20:45:38.964000+00:00,0,0
2,3049202,Duloxetine,3,2020-06-20 12:26:00+00:00,2020-06-20 12:26:00+00:00,0,0
3,3049202,Venlafaxine,1,2008-03-01 19:59:00.040000+00:00,2008-03-01 19:59:00.040000+00:00,0,0
4,3049202,Venlafaxine,2,2009-09-17 04:06:47.500000+00:00,2009-09-17 04:06:47.500000+00:00,0,0


In [31]:
# Function to compute other antidepressant columns
def compute_other_ad(row, person_counts):
    # If only one row, then there is no history of other antidepressants
    if person_counts[row['person_id']] == 1:
        return 0
    else:
        # Calculate the trail start and end dates
        trail_start = row['earliest_ad_start']
        trail_end = trail_start + pd.Timedelta(days=70)
        
        # Check other rows for the same person_id for other antidepressants
        other_rows = ad_dates_df[(ad_dates_df['person_id'] == row['person_id']) &
                                  (ad_dates_df['ad_grouping'] != row['ad_grouping'])]
        
        # Find rows where antidepressants started within the trail period; return 1 if found
        for _, other_row in other_rows.iterrows():
            ad_start = other_row['earliest_ad_start']
            if pd.notna(ad_start) and trail_start <= ad_start <= trail_end:
                return 1
        
        # Return 0 if no other antidepressants were found within the trail period
        return 0


In [32]:
#Count the occurence of each person_id
person_counts = ad_dates_df['person_id'].value_counts()

#Fill values for the use of other antidepressants for the time period
ad_dates_df['other_ad'] = ad_dates_df.apply(compute_other_ad, axis=1, args=(person_counts,))

ad_dates_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0
1,3049202,Duloxetine,2,2019-06-23 20:45:38.964000+00:00,2019-06-23 20:45:38.964000+00:00,0,0,0
2,3049202,Duloxetine,3,2020-06-20 12:26:00+00:00,2020-06-20 12:26:00+00:00,0,0,0
3,3049202,Venlafaxine,1,2008-03-01 19:59:00.040000+00:00,2008-03-01 19:59:00.040000+00:00,0,0,1
4,3049202,Venlafaxine,2,2009-09-17 04:06:47.500000+00:00,2009-09-17 04:06:47.500000+00:00,0,0,0


In [33]:
print(ad_dates_df.columns)

Index(['person_id', 'ad_grouping', 'trail_number', 'earliest_ad_start',
       'earliest_ad_end', 'dosage_days', 'dose_70p', 'other_ad'],
      dtype='object')


In [34]:
import numpy as np

# If dose_70p is 1 and other_ad is 0 for the trail, then Remission is 1 otherwise 0
ad_dates_df['Remission'] = np.where(
    (ad_dates_df["dose_70p"] == 1) & (ad_dates_df["other_ad"] == 0), 
    1, 
    0
)

# Display the first few rows
print(ad_dates_df.head())


   person_id  ad_grouping  trail_number                earliest_ad_start  \
0    3049202   Duloxetine             1 2018-12-23 21:25:59.429000+00:00   
1    3049202   Duloxetine             2 2019-06-23 20:45:38.964000+00:00   
2    3049202   Duloxetine             3        2020-06-20 12:26:00+00:00   
3    3049202  Venlafaxine             1 2008-03-01 19:59:00.040000+00:00   
4    3049202  Venlafaxine             2 2009-09-17 04:06:47.500000+00:00   

                   earliest_ad_end  dosage_days  dose_70p  other_ad  Remission  
0 2018-12-23 21:25:59.429000+00:00            0         0         0          0  
1 2019-06-23 20:45:38.964000+00:00            0         0         0          0  
2        2020-06-20 12:26:00+00:00            0         0         0          0  
3 2008-03-01 19:59:00.040000+00:00            0         0         1          0  
4 2009-09-17 04:06:47.500000+00:00            0         0         0          0  


### CREATE ANALYSIS DATAFRAME 

In [35]:
#Create analysis dataframe by left joining with demographics dataframe and then the dead dataframe
analysis_df = pd.merge(ad_dates_df, demographics_df, on='person_id', how= 'left')

#Rename column to date_of_death in dead_df
dead_df = dead_df.rename(columns={'observation_datetime': 'date_of_death'})

#left join with analysis_df on person_id
analysis_df = pd.merge(analysis_df, dead_df, on='person_id', how='left')

analysis_df.head()


,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad,Remission,gender_concept_id,gender,date_of_birth,race_concept_id,race,ethnicity_concept_id,ethnicity,sex_at_birth_concept_id,sex_at_birth,date_of_death
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,Female,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT
1,3049202,Duloxetine,2,2019-06-23 20:45:38.964000+00:00,2019-06-23 20:45:38.964000+00:00,0,0,0,0,45878463,Female,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT
2,3049202,Duloxetine,3,2020-06-20 12:26:00+00:00,2020-06-20 12:26:00+00:00,0,0,0,0,45878463,Female,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT
3,3049202,Venlafaxine,1,2008-03-01 19:59:00.040000+00:00,2008-03-01 19:59:00.040000+00:00,0,0,1,0,45878463,Female,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT
4,3049202,Venlafaxine,2,2009-09-17 04:06:47.500000+00:00,2009-09-17 04:06:47.500000+00:00,0,0,0,0,45878463,Female,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT


In [36]:
#Set data types that are not appropriate after creation of new dataframe

#Dates should be correctly typed
analysis_df['earliest_ad_start'] = pd.to_datetime(analysis_df['earliest_ad_start'])
analysis_df['earliest_ad_end'] = pd.to_datetime(analysis_df['earliest_ad_end'])

#Print the Datafarme with updated datatypes
print(analysis_df.dtypes)

person_id                                int64
ad_grouping                             object
trail_number                             int64
earliest_ad_start          datetime64[ns, UTC]
earliest_ad_end            datetime64[ns, UTC]
dosage_days                              int64
dose_70p                                 int64
other_ad                                 int64
Remission                                int64
gender_concept_id                        Int64
gender                                  object
date_of_birth              datetime64[ns, UTC]
race_concept_id                          Int64
race                                    object
ethnicity_concept_id                     Int64
ethnicity                               object
sex_at_birth_concept_id                  Int64
sex_at_birth                            object
date_of_death              datetime64[ns, UTC]
dtype: object


#### CALCULATE AGE AT ANTIDEPRESAANTS START AND AGE AT DEATH 

In [37]:
#Calculate age in years at adantidepressant start
analysis_df['age_at_ad'] = (analysis_df['earliest_ad_start'] - analysis_df ['date_of_birth']).dt.days / 365.25

#For 'age_at_death, only calculate where 'date_of_death' is not null
#this ensures we don't get NaN values for people who are still alive
analysis_df.loc[analysis_df['date_of_death'].notnull(), 'age_at_death'] = (
  analysis_df['date_of_death'] - analysis_df['date_of_birth']).dt.days / 365.25

analysis_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad,Remission,gender_concept_id,...,date_of_birth,race_concept_id,race,ethnicity_concept_id,ethnicity,sex_at_birth_concept_id,sex_at_birth,date_of_death,age_at_ad,age_at_death
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT,61.415469,NaN
1,3049202,Duloxetine,2,2019-06-23 20:45:38.964000+00:00,2019-06-23 20:45:38.964000+00:00,0,0,0,0,45878463,...,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT,61.913758,NaN
2,3049202,Duloxetine,3,2020-06-20 12:26:00+00:00,2020-06-20 12:26:00+00:00,0,0,0,0,45878463,...,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT,62.907598,NaN
3,3049202,Venlafaxine,1,2008-03-01 19:59:00.040000+00:00,2008-03-01 19:59:00.040000+00:00,0,0,1,0,45878463,...,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT,50.603696,NaN
4,3049202,Venlafaxine,2,2009-09-17 04:06:47.500000+00:00,2009-09-17 04:06:47.500000+00:00,0,0,0,0,45878463,...,1957-07-24 00:00:00+00:00,2100000001,None Indicated,38003563,Hispanic or Latino,45878463,Female,NaT,52.150582,NaN


#### CREATE AGE GROUPS 

Age groups will be as follows:
* 0 to 12
* 13 to 19
* 20 to 40
* 41 to 64
* 65 to 79
* 80 to 89


In [38]:
#Get the age range in the data
#Find the minimum age at ad
min_age_at_ad = analysis_df['age_at_ad'].min()

#Find the maximum age at ad
max_age_at_ad = analysis_df['age_at_ad'].max()

print(f"Minimum age at ad: {min_age_at_ad}")
print(f"Maximum age at ad: {max_age_at_ad}")

Minimum age at ad: 9.360711841204655
Maximum age at ad: 87.5564681724846


In [39]:
#Define bins for age ranges
bins = [0, 12, 19, 40, 64, 79, 89]

#Define labels for age groups
labels = ['age 0-12', 'age 13-19', 'age 20-40', 'age 41-64', 'age 65-79', 'age 80-89']

#Create a categorical column for age ranges
analysis_df['age_range'] = pd.cut(analysis_df['age_at_ad'], bins=bins, labels=labels, right=True, include_lowest=True)

#Create dummy variables for the age ranges
age_dummies = pd.get_dummies(analysis_df['age_range']).astype(int)

#Concatenate the dummy variables with the original Dataframe
analysis_df = pd.concat([analysis_df, age_dummies], axis=1)

analysis_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad,Remission,gender_concept_id,...,date_of_death,age_at_ad,age_at_death,age_range,age 0-12,age 13-19,age 20-40,age 41-64,age 65-79,age 80-89
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,NaT,61.415469,NaN,age 41-64,0,0,0,1,0,0
1,3049202,Duloxetine,2,2019-06-23 20:45:38.964000+00:00,2019-06-23 20:45:38.964000+00:00,0,0,0,0,45878463,...,NaT,61.913758,NaN,age 41-64,0,0,0,1,0,0
2,3049202,Duloxetine,3,2020-06-20 12:26:00+00:00,2020-06-20 12:26:00+00:00,0,0,0,0,45878463,...,NaT,62.907598,NaN,age 41-64,0,0,0,1,0,0
3,3049202,Venlafaxine,1,2008-03-01 19:59:00.040000+00:00,2008-03-01 19:59:00.040000+00:00,0,0,1,0,45878463,...,NaT,50.603696,NaN,age 41-64,0,0,0,1,0,0
4,3049202,Venlafaxine,2,2009-09-17 04:06:47.500000+00:00,2009-09-17 04:06:47.500000+00:00,0,0,0,0,45878463,...,NaT,52.150582,NaN,age 41-64,0,0,0,1,0,0


#### Indicator for female at the time of birth

In [40]:
analysis_df['Female'] = np.where(analysis_df['gender'] == 'Female', 1, 0)
analysis_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad,Remission,gender_concept_id,...,age_at_ad,age_at_death,age_range,age 0-12,age 13-19,age 20-40,age 41-64,age 65-79,age 80-89,Female
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,61.415469,NaN,age 41-64,0,0,0,1,0,0,1
1,3049202,Duloxetine,2,2019-06-23 20:45:38.964000+00:00,2019-06-23 20:45:38.964000+00:00,0,0,0,0,45878463,...,61.913758,NaN,age 41-64,0,0,0,1,0,0,1
2,3049202,Duloxetine,3,2020-06-20 12:26:00+00:00,2020-06-20 12:26:00+00:00,0,0,0,0,45878463,...,62.907598,NaN,age 41-64,0,0,0,1,0,0,1
3,3049202,Venlafaxine,1,2008-03-01 19:59:00.040000+00:00,2008-03-01 19:59:00.040000+00:00,0,0,1,0,45878463,...,50.603696,NaN,age 41-64,0,0,0,1,0,0,1
4,3049202,Venlafaxine,2,2009-09-17 04:06:47.500000+00:00,2009-09-17 04:06:47.500000+00:00,0,0,0,0,45878463,...,52.150582,NaN,age 41-64,0,0,0,1,0,0,1


### PREPARE DISEASES DATAFRAME 

#### REMOVE DUPLICATES

In [41]:
diseases_df.shape[0] #number of rows with duplicates

3976892

In [42]:
#Remove duplicate rows
diseases_df = diseases_df.drop_duplicates(
     subset=['person_id', 'standard_concept_code', 'condition_start_datetime', 'condition_end_datetime'])

In [43]:
diseases_df.shape[0] #number of rows without duplicates

3314518

In [44]:
diseases_df.head()

,person_id,condition_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,condition_start_datetime,condition_end_datetime,condition_type_concept_id,condition_type_concept_name,stop_reason,visit_occurrence_id,visit_occurrence_concept_name,condition_source_value,condition_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,condition_status_source_value,condition_status_concept_id,condition_status_concept_name
0,2179796,43531578,Chronic kidney disease due to type 2 diabetes ...,771000119108,SNOMED,2018-09-01 04:00:00+00:00,NaT,32821,EHR billing record,None,<NA>,None,E11.22,45595797,Type 2 diabetes mellitus with diabetic chronic...,E11.22,ICD10CM,None,<NA>,None
1,1622769,81169,Superficial foreign body of finger without maj...,52877004,SNOMED,1997-01-14 00:00:00+00:00,NaT,44786629,Secondary Condition,None,27000000010091692,None,915.6,44830569,Superficial foreign body (splinter) of finger(...,915.6,ICD9CM,None,<NA>,None
2,1557758,4149904,"Attention deficit hyperactivity disorder, comb...",31177006,SNOMED,2022-01-14 00:00:00+00:00,NaT,44786627,Primary Condition,None,44000000000448720,None,F90.2,45552506,"Attention-deficit hyperactivity disorder, comb...",F90.2,ICD10CM,None,<NA>,None
3,2172387,380111,Acute atopic conjunctivitis,67678004,SNOMED,2009-10-04 00:00:00+00:00,NaT,44786627,Primary Condition,None,27000000007016720,None,372.05,44823068,Acute atopic conjunctivitis,372.05,ICD9CM,None,<NA>,None
4,3152137,4332246,Aneurysm,432119003,SNOMED,2007-09-26 04:00:00+00:00,NaT,32821,EHR billing record,None,<NA>,None,437.3,44826646,"Cerebral aneurysm, nonruptured",437.3,ICD9CM,None,<NA>,None


#### Combine with analysis dataframe 

In [45]:
analysis_df = pd.merge(analysis_df, diseases_df, on='person_id', how='left')
analysis_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad,Remission,gender_concept_id,...,visit_occurrence_id,visit_occurrence_concept_name,condition_source_value,condition_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,condition_status_source_value,condition_status_concept_id,condition_status_concept_name
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,<NA>,None,296.24,44820717,"Major depressive affective disorder, single ep...",296.24,ICD9CM,None,<NA>,None
1,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,<NA>,None,F33.3,35207159,"Major depressive disorder, recurrent, severe w...",F33.3,ICD10CM,None,<NA>,None
2,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,<NA>,None,311,44833421,"Depressive disorder, not elsewhere classified",311,ICD9CM,None,<NA>,None
3,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,33000000016944920,Outpatient Visit,F17.210,45562025,"Nicotine dependence, cigarettes, uncomplicated",F17.210,ICD10CM,None,<NA>,None
4,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,33000000002945864,Outpatient Visit,R31.29,37201088,Other microscopic hematuria,R31.29,ICD10CM,None,<NA>,None


#### Select only diseases that started prior to starting the antidepressant before each trial  

In [46]:
#number of rows prior to cleanup
analysis_df.shape[0]

17695390

In [47]:
#Retain only rows where condition_start_datetime is less than earliest_ad_start
analysis_df = analysis_df[analysis_df['condition_start_datetime'] < analysis_df['earliest_ad_start']]

#Reset index
analysis_df.reset_index(drop=True, inplace=True)

In [48]:
#number of rows after to cleanup
analysis_df.shape[0]

6330003

In [49]:
analysis_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad,Remission,gender_concept_id,...,visit_occurrence_id,visit_occurrence_concept_name,condition_source_value,condition_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,condition_status_source_value,condition_status_concept_id,condition_status_concept_name
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,<NA>,None,296.24,44820717,"Major depressive affective disorder, single ep...",296.24,ICD9CM,None,<NA>,None
1,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,<NA>,None,311,44833421,"Depressive disorder, not elsewhere classified",311,ICD9CM,None,<NA>,None
2,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,33000000016944920,Outpatient Visit,F17.210,45562025,"Nicotine dependence, cigarettes, uncomplicated",F17.210,ICD10CM,None,<NA>,None
3,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,<NA>,None,715.91,44823350,"Osteoarthrosis, unspecified whether generalize...",715.91,ICD9CM,None,<NA>,None
4,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,33000000023454684,Outpatient Visit,455.6,44831267,Unspecified hemorrhoids without mention of com...,455.6,ICD9CM,None,<NA>,None


## DATA CHECKPOINT 

### STORE THE ANALYSIS IN A DATAFRAME 

In [50]:
#Check data types prior to writing file for reference after reading
analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6330003 entries, 0 to 6330002
Data columns (total 48 columns):
 #   Column                         Dtype              
---  ------                         -----              
 0   person_id                      int64              
 1   ad_grouping                    object             
 2   trail_number                   int64              
 3   earliest_ad_start              datetime64[ns, UTC]
 4   earliest_ad_end                datetime64[ns, UTC]
 5   dosage_days                    int64              
 6   dose_70p                       int64              
 7   other_ad                       int64              
 8   Remission                      int64              
 9   gender_concept_id              Int64              
 10  gender                         object             
 11  date_of_birth                  datetime64[ns, UTC]
 12  race_concept_id                Int64              
 13  race                           object     

In [51]:
analysis_df.to_csv("./analysis_file.csv" , index = False)

In [52]:
import pandas as pd 
import numpy as np
import os 

In [53]:
analysis_df = pd.read_csv("./analysis_file.csv" , low_memory = False)

In [54]:
#Check data types after loading from file for reference on which datatypes to change 
analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6330003 entries, 0 to 6330002
Data columns (total 48 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   person_id                      int64  
 1   ad_grouping                    object 
 2   trail_number                   int64  
 3   earliest_ad_start              object 
 4   earliest_ad_end                object 
 5   dosage_days                    int64  
 6   dose_70p                       int64  
 7   other_ad                       int64  
 8   Remission                      int64  
 9   gender_concept_id              int64  
 10  gender                         object 
 11  date_of_birth                  object 
 12  race_concept_id                int64  
 13  race                           object 
 14  ethnicity_concept_id           int64  
 15  ethnicity                      object 
 16  sex_at_birth_concept_id        int64  
 17  sex_at_birth                   object 
 18  da

In [55]:
#Change datatypes to aid processing 
#integer columns (imported correctly)
#['person_id' , 'trial_number' , 'dosage_days' , 'dose_70p' 'other_ad' , 'Remission', 'gender_concept_id',
#'ethnicity_concept_id' , 'sex_at_birth_concept_id', 'age 0-12' , 'age 13-19' , 'age 20-40','age 41-64' , 'age 65-79' , 
#'age 80-89', 'Female', 'disease_group']

#Float columns (improved correctly)
#['age_at_ad', 'age_at_death']

#Date columns 
dates_col = ['earliest_ad_start', 'earliest_ad_end', 'date_of_birth', 'date_of_death',
            'condition_start_datetime', 'condition_end_datetime']

for col in dates_col:
    analysis_df[col] = pd.to_datetime(analysis_df[col], errors='coerce')
    

#Rename columns for diseases 
analysis_df = analysis_df.rename(columns={'standard_concept_name': 'disease_name'})
analysis_df = analysis_df.rename(columns={'standard_concept_code': 'disease_code'})

In [56]:
analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6330003 entries, 0 to 6330002
Data columns (total 48 columns):
 #   Column                         Dtype              
---  ------                         -----              
 0   person_id                      int64              
 1   ad_grouping                    object             
 2   trail_number                   int64              
 3   earliest_ad_start              datetime64[ns, UTC]
 4   earliest_ad_end                datetime64[ns, UTC]
 5   dosage_days                    int64              
 6   dose_70p                       int64              
 7   other_ad                       int64              
 8   Remission                      int64              
 9   gender_concept_id              int64              
 10  gender                         object             
 11  date_of_birth                  datetime64[ns, UTC]
 12  race_concept_id                int64              
 13  race                           object     

In [57]:
analysis_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,earliest_ad_end,dosage_days,dose_70p,other_ad,Remission,gender_concept_id,...,visit_occurrence_id,visit_occurrence_concept_name,condition_source_value,condition_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,condition_status_source_value,condition_status_concept_id,condition_status_concept_name
0,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,NaN,NaN,296.24,44820717.0,"Major depressive affective disorder, single ep...",296.24,ICD9CM,NaN,NaN,NaN
1,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,NaN,NaN,311,44833421.0,"Depressive disorder, not elsewhere classified",311,ICD9CM,NaN,NaN,NaN
2,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,3.300000e+16,Outpatient Visit,F17.210,45562025.0,"Nicotine dependence, cigarettes, uncomplicated",F17.210,ICD10CM,NaN,NaN,NaN
3,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,NaN,NaN,715.91,44823350.0,"Osteoarthrosis, unspecified whether generalize...",715.91,ICD9CM,NaN,NaN,NaN
4,3049202,Duloxetine,1,2018-12-23 21:25:59.429000+00:00,2018-12-23 21:25:59.429000+00:00,0,0,0,0,45878463,...,3.300000e+16,Outpatient Visit,455.6,44831267.0,Unspecified hemorrhoids without mention of com...,455.6,ICD9CM,NaN,NaN,NaN


### Reduce the dimensionality

In [58]:
cols_to_keep = ['person_id',
               'ad_grouping',
               'trail_number',
               'earliest_ad_start',
               'Remission',
               'age 13-19',
               'age 20-40',
               'age 41-64',
               'age 65-79',
               'age 80-89',
               'Female',
               'disease_code'
               ]

#select only the columns in cols_to_keep
analysis_df_reduced = analysis_df[cols_to_keep]

analysis_df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6330003 entries, 0 to 6330002
Data columns (total 12 columns):
 #   Column             Dtype              
---  ------             -----              
 0   person_id          int64              
 1   ad_grouping        object             
 2   trail_number       int64              
 3   earliest_ad_start  datetime64[ns, UTC]
 4   Remission          int64              
 5   age 13-19          int64              
 6   age 20-40          int64              
 7   age 41-64          int64              
 8   age 65-79          int64              
 9   age 80-89          int64              
 10  Female             int64              
 11  disease_code       int64              
dtypes: datetime64[ns, UTC](1), int64(10), object(1)
memory usage: 579.5+ MB


### Convert disease_codes into dummy variables in store in a new copy of dataframes

In [59]:
analysis_df_reduced['disease_code'].isnull().sum()

0

In [60]:
# Loop through each unique antidepressant to create an analysis dataframe for each
for ad in analysis_df_reduced['ad_grouping'].unique():
    # Create a subset for the antidepressant
    analysis_subset = analysis_df_reduced[analysis_df_reduced['ad_grouping'] == ad]

    # Check columns in analysis_subset
    print(f"Columns in analysis_subset for {ad}:", analysis_subset.columns)

    # Create a copy containing only primary key columns and disease code to create dummy variables 
    disease_per_trial = analysis_subset[['person_id', 'ad_grouping', 'trail_number', 'disease_code']].copy()  # changed 'trail_number' to 'trial_number'
    
    # Create dummy variables for the disease codes
    disease_dummies = pd.get_dummies(disease_per_trial['disease_code'], prefix='disease').astype(int)
    
    # Concatenate the dummy variables with the disease per trial dataframe
    disease_per_trial = pd.concat([disease_per_trial, disease_dummies], axis=1)
    
    # Drop the disease_code column in disease per trial
    disease_per_trial = disease_per_trial.drop(columns=['disease_code'])
    
    # Drop the disease code column from the analysis subset dataframe
    analysis_subset = analysis_subset.drop(columns=['disease_code'])
    
    # Drop columns with less than 30 samples as they do not have enough data to properly identify distributions 
    # Take column sums 
    columns_sums = disease_per_trial.sum()
    
    columns_to_drop = [col for col in disease_per_trial.columns if col.startswith('disease') and columns_sums[col] < 30]

    # Drop columns with less than 30 samples
    disease_per_trial = disease_per_trial.drop(columns=columns_to_drop)
    
    # Aggregate analysis_df_reduced based on primary keys, use first value for aggregation 
    aggregation_dict_1 = {col: 'first'
                           for col in analysis_subset.columns if col not in ['person_id', 'ad_grouping', 'trail_number']}
    
    # Group by the primary key columns and aggregate
    analysis_subset = analysis_subset.groupby(['person_id', 'ad_grouping', 'trail_number'],
                                             as_index=False).agg(aggregation_dict_1)
                                              
    # Aggregate disease per trial based on primary keys, use max value for aggregation   
    aggregation_dict_2 = {col: 'max'
                           for col in disease_per_trial.columns if col not in ['person_id', 'ad_grouping', 'trail_number']}
                                              
    # Group by the primary key columns and aggregate
    disease_per_trial = disease_per_trial.groupby(['person_id', 'ad_grouping', 'trail_number'],
                                                 as_index=False).agg(aggregation_dict_2)
   
    # Left join analysis_subset and disease_per_trial using primary keys 
    analysis_subset = pd.merge(analysis_subset, disease_per_trial, on=['person_id', 'ad_grouping', 'trail_number'],
                               how='left')
              
    # Create files for each subset 
    filename = f'./analysis_subset_{ad}.csv'
    analysis_subset.to_csv(filename, index=False)


Columns in analysis_subset for Duloxetine: Index(['person_id', 'ad_grouping', 'trail_number', 'earliest_ad_start',
       'Remission', 'age 13-19', 'age 20-40', 'age 41-64', 'age 65-79',
       'age 80-89', 'Female', 'disease_code'],
      dtype='object')
Columns in analysis_subset for Venlafaxine: Index(['person_id', 'ad_grouping', 'trail_number', 'earliest_ad_start',
       'Remission', 'age 13-19', 'age 20-40', 'age 41-64', 'age 65-79',
       'age 80-89', 'Female', 'disease_code'],
      dtype='object')
Columns in analysis_subset for Nortriptyline: Index(['person_id', 'ad_grouping', 'trail_number', 'earliest_ad_start',
       'Remission', 'age 13-19', 'age 20-40', 'age 41-64', 'age 65-79',
       'age 80-89', 'Female', 'disease_code'],
      dtype='object')
Columns in analysis_subset for Mirtazapine: Index(['person_id', 'ad_grouping', 'trail_number', 'earliest_ad_start',
       'Remission', 'age 13-19', 'age 20-40', 'age 41-64', 'age 65-79',
       'age 80-89', 'Female', 'disease_c

In [61]:
import pandas as pd 
import numpy as np 
import os

In [62]:
ad_list = ['Amitriptyline',
          'Bupropion',
          'Citalopram',
          'Desvenlafaxine',
          'Doxepin',
          'Duloxetine',
          'Escitalopram',
          'Fluoxetine',
           'Mirtazapine',
           'Nortriptyline',
           'Paroxetine',
           'Sertraline',
           'Trazodone',
           'Venlafaxine',
           'Other']

analysis_df_dict = dict()
for ad in ad_list:
    filename = f'./analysis_subset_{ad}.csv'
    analysis_df_dict[ad] = pd.read_csv(filename, low_memory = False)

In [63]:
#check the info and head of each file 
for ad in ad_list:
    print(f"\n\nAntidepressants: {ad}")
    print(analysis_df_dict[ad].info())
    print(analysis_df_dict[ad].head())



Antidepressants: Amitriptyline
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1991 entries, 0 to 1990
Columns: 1440 entries, person_id to disease_15960141000119102
dtypes: int64(1438), object(2)
memory usage: 21.9+ MB
None
   person_id    ad_grouping  trail_number                 earliest_ad_start  \
0    1000109  Amitriptyline             1                               NaN   
1    1000291  Amitriptyline             1  2017-05-22 16:07:37.600000+00:00   
2    1000291  Amitriptyline             2  2017-07-11 13:02:01.557000+00:00   
3    1000291  Amitriptyline             3                               NaN   
4    1000951  Amitriptyline             1                               NaN   

   Remission  age 13-19  age 20-40  age 41-64  age 65-79  age 80-89  ...  \
0          0          0          0          1          0          0  ...   
1          0          0          0          1          0          0  ...   
2          0          0          0          1          0          0  

In [64]:
## Remove disease codes with less than 30 observations
disease_counts = analysis_df_reduced['disease_code'].value_counts()
mask = analysis_df_reduced['disease_code'].isin(disease_counts[disease_counts >= 30].index)
analysis_df_reduced = analysis_df_reduced[mask]
analysis_df_reduced.reset_index(drop=True, inplace=True)



In [65]:
print(analysis_subset.columns)


Index(['person_id', 'ad_grouping', 'trail_number', 'earliest_ad_start',
       'Remission', 'age 13-19', 'age 20-40', 'age 41-64', 'age 65-79',
       'age 80-89',
       ...
       'disease_1085861000119107', 'disease_1085871000119101',
       'disease_1085911000119103', 'disease_1085991000119107',
       'disease_1092861000119101', 'disease_10633391000119104',
       'disease_10641391000119104', 'disease_10685111000119102',
       'disease_10742471000119108', 'disease_10743881000119104'],
      dtype='object', length=1245)


In [66]:
# Define the antidepressant you want to analyze, e.g., Sertraline
ad = 'Sertraline'

# Subset for the selected antidepressant
analysis_subset = analysis_df_reduced[analysis_df_reduced['ad_grouping'] == ad]

# Remove disease codes with less than 30 observations before dummy creation to reduce rows
disease_counts = analysis_subset['disease_code'].value_counts()
mask = analysis_subset['disease_code'].isin(disease_counts[disease_counts >= 30].index)
analysis_subset = analysis_subset[mask]
analysis_subset.reset_index(drop=True, inplace=True)

# Create a copy containing only primary key columns and disease code to create dummy variables
disease_per_trial = analysis_subset[['person_id', 'ad_grouping', 'trail_number', 'disease_code']].copy()

# Create dummy variables for the disease codes
disease_dummies = pd.get_dummies(disease_per_trial['disease_code'], prefix='disease').astype(int)

# Concatenate the dummy variables with the disease per trial dataframe
disease_per_trial = pd.concat([disease_per_trial, disease_dummies], axis=1)

# Drop the disease_code column in disease per trial
disease_per_trial = disease_per_trial.drop(columns=['disease_code'])

# Drop the disease_code column from the analysis subset dataframe
analysis_subset = analysis_subset.drop(columns=['disease_code'])

# Aggregate analysis_df_reduced based on primary keys, use first value for aggregation
aggregation_dict_1 = {col: 'first' 
                      for col in analysis_subset.columns if col not in ['person_id', 'ad_grouping', 'trail_number']}

# Group by the primary key columns and aggregate
analysis_subset = analysis_subset.groupby(['person_id', 'ad_grouping', 'trail_number'], 
                                          as_index=False).agg(aggregation_dict_1)

# Aggregate disease per trial based on primary keys, use max value for aggregation
aggregation_dict_2 = {col: 'max' 
                      for col in disease_per_trial.columns if col not in ['person_id', 'ad_grouping', 'trail_number']}

# Group by the primary key columns and aggregate
disease_per_trial = disease_per_trial.groupby(['person_id', 'ad_grouping', 'trail_number'], 
                                              as_index=False).agg(aggregation_dict_2)

# Left merge analysis_df_reduced and disease_per_trial using primary keys (analysis_df_reduced on the left)
analysis_subset = pd.merge(analysis_subset, disease_per_trial, 
                           on=['person_id', 'ad_grouping', 'trail_number'], how='left')

# Create a checkpoint file for this subset
filename = f'./checkpoint/hap786_analysis_subset_{ad}.csv'
analysis_subset.to_csv(filename, index=False)
# Create a function to get the antidepressant history for one specific drug
def get_ad_hist(row):
    
    # If it's the first trial, return 0 (no history)
    if row['trial_number'] == 1:
        return 0

    # Check previous trials of the same antidepressant
    previous_trials = ad_dates_df[(ad_dates_df['person_id'] == row['person_id']) & 
                                  (ad_dates_df['trial_number'] < row['trial_number']) &
                                  (ad_dates_df['ad_grouping'] == row['ad_grouping'])]

    # Check if there are any previous trials with no remission
    if previous_trials[previous_trials['Remission'] == 0].empty:
        return 0  # No history of non-remission
    else:
        return 1  # History of non-remission found

print(f"Checkpoint file created for {ad}")


Checkpoint file created for Sertraline


In [71]:
# Function to get the antidepressant history (SN, SR, DN, DR)
def get_ad_hist(row, hist_type):
    # If it's the first trial and SN or SR, return 0 (no history)
    if row['trail_number'] == 1 and hist_type in ['SN', 'SR']:
        return 0

    # If processing for SN or SR
    if hist_type in ['SN', 'SR']:
        previous_trials = ad_dates_df[
            (ad_dates_df['person_id'] == row['person_id']) & 
            (ad_dates_df['trail_number'] < row['trail_number']) &
            (ad_dates_df['ad_grouping'] == row['ad_grouping'])
        ]
        if hist_type == 'SN':
            # Check for any previous trial with no remission
            return 1 if not previous_trials[previous_trials['Remission'] == 0].empty else 0
        else:  # SR case
            # Check for any previous trial with remission
            return 1 if not previous_trials[previous_trials['Remission'] == 1].empty else 0
    # If processing for DN or DR
    elif hist_type in ['DN', 'DR']:
        trial_start = row['earliest_ad_start']
        other_rows = ad_dates_df[
            (ad_dates_df['person_id'] == row['person_id']) & 
            (ad_dates_df['ad_grouping'] != row['ad_grouping']) &
            (ad_dates_df['earliest_ad_end'] < trial_start)
        ]
        if hist_type == 'DN':
            return 1 if not other_rows[other_rows['Remission'] == 0].empty else 0
        else:  # DR case
            return 1 if not other_rows[other_rows['Remission'] == 1].empty else 0
    return 0

In [72]:

# Add these columns to the dataset
for hist_type in ['SN', 'SR', 'DN', 'DR']:
    analysis_subset[hist_type] = analysis_subset.apply(get_ad_hist, axis=1, args=(hist_type,))


In [73]:
# Get episode counts
def get_episodes_count(row):
    previous_episodes = ad_dates_df[
        (ad_dates_df['person_id'] == row['person_id']) & 
        (ad_dates_df['earliest_ad_end'] < row['earliest_ad_start'])
    ]
    if previous_episodes.empty:
        return 0
    previous_episodes = previous_episodes.sort_values(by='earliest_ad_start')
    episode_count, current_end_date = 1, previous_episodes.iloc[0]['earliest_ad_end']
    
    for index, previous_row in previous_episodes.iterrows():
        if previous_row['earliest_ad_start'] > current_end_date:
            episode_count += 1
        current_end_date = max(current_end_date, previous_row['earliest_ad_end'])
    return episode_count

In [74]:
# Add episode count column
analysis_subset['num_episodes'] = analysis_subset.apply(get_episodes_count, axis=1)


In [75]:
# Add remission count
def get_remission_count(row):
    previous_episodes = ad_dates_df[
        (ad_dates_df['person_id'] == row['person_id']) & 
        (ad_dates_df['earliest_ad_end'] < row['earliest_ad_start'])
    ]
    return previous_episodes['Remission'].sum() if not previous_episodes.empty else 0


In [76]:
# Add remission count column
analysis_subset['num_remissions'] = analysis_subset.apply(get_remission_count, axis=1)


In [78]:
# Add antidepressant count
def get_ad_count(row):
    previous_episodes = ad_dates_df[
        (ad_dates_df['person_id'] == row['person_id']) & 
        (ad_dates_df['earliest_ad_end'] < row['earliest_ad_start'])
    ]
    return previous_episodes['ad_grouping'].nunique() if not previous_episodes.empty else 0


In [79]:
 #Add number of antidepressants column
analysis_subset['num_antidepressants'] = analysis_subset.apply(get_ad_count, axis=1)



In [80]:
# Dummy variable creation based on number of episodes, remissions, and antidepressants
analysis_subset['epi_2p'] = (analysis_subset['num_episodes'] >= 2).astype(int)
analysis_subset['rem_2p'] = (analysis_subset['num_remissions'] >= 2).astype(int)
analysis_subset['adep_23'] = ((analysis_subset['num_antidepressants'] >= 2) & (analysis_subset['num_antidepressants'] <= 3)).astype(int)
analysis_subset['adep_4p'] = (analysis_subset['num_antidepressants'] >= 4).astype(int)


In [81]:
# Save the analysis_subset to a CSV for checkpoint
filename = f'./checkpoint/hap786_analysis_with_history_{ad}.csv'
analysis_subset.to_csv(filename, index=False)

print(f"Updated analysis for {ad} saved to {filename}")

Updated analysis for Sertraline saved to ./checkpoint/hap786_analysis_with_history_Sertraline.csv


In [82]:
# Now, load the updated analysis subset and proceed with regression steps
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LassoCV, LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, log_loss
import numpy as np

# Suppress convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Reload the dataset if necessary
filename = f'./checkpoint/hap786_analysis_with_history_{ad}.csv'
analysis_df = pd.read_csv(filename)



In [83]:
analysis_df.head()

,person_id,ad_grouping,trail_number,earliest_ad_start,Remission,age 13-19,age 20-40,age 41-64,age 65-79,age 80-89,...,SR,DN,DR,num_episodes,num_remissions,num_antidepressants,epi_2p,rem_2p,adep_23,adep_4p
0,1000109,Sertraline,1,NaN,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000291,Sertraline,1,2014-06-22 15:26:41.600000+00:00,0,0,0,1,0,0,...,0,1,0,1,0,1,0,0,0,0
2,1000291,Sertraline,2,2015-01-27 16:13:56.600000+00:00,0,0,0,1,0,0,...,0,1,0,2,0,2,1,0,1,0
3,1000291,Sertraline,3,2015-04-21 19:18:07.200000+00:00,0,0,0,1,0,0,...,0,1,0,3,0,2,1,0,1,0
4,1000951,Sertraline,1,NaN,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# Ensure only numeric columns are used for correlation calculation
numeric_columns = analysis_df.select_dtypes(include=[np.number])

# Check if any numeric columns exist before proceeding
if not numeric_columns.empty:
    # Calculate correlation matrix for numeric columns only
    correlation_matrix = numeric_columns.corr()
    
    # Proceed with removing multicollinear columns
    multicollinear_columns = set()
    
    for i in range(len(correlation_matrix.columns)):
        for j in range(i + 1, len(correlation_matrix.columns)):
            if correlation_matrix.iloc[i, j] == 1.0:
                column_name_i = correlation_matrix.columns[i]
                multicollinear_columns.add(column_name_i)
    
    # Drop multicollinear columns from the analysis_df
    regression_df = analysis_df.drop(columns=list(multicollinear_columns)).copy()

In [101]:
# Check the distribution of the Remission target variable
class_distribution = y.value_counts()
print(class_distribution)


Remission
0    3908
Name: count, dtype: int64


In [111]:
# Check the distribution of the 'Remission' column
class_distribution = analysis_df['Remission'].value_counts()
print(class_distribution)

# Check the number of samples in the balanced dataset
print(f"Number of samples in balanced dataset: {X_balanced.shape[0]}")

# Verify the class distribution in the balanced dataset
print(f"Class distribution after balancing: \n{y_balanced.value_counts()}")


Remission
0    3908
Name: count, dtype: int64
Number of samples in balanced dataset: 0
Class distribution after balancing: 
Series([], Name: count, dtype: int64)


In [115]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, log_loss
from sklearn.exceptions import NotFittedError

# Number of samples to added (based on the imbalance)
num_new_samples = 100  

# Generate random data for new samples based on the features in X
new_samples = np.random.randn(num_new_samples, X.shape[1])
new_labels = np.ones(num_new_samples)  # Label the new samples as 'Remission = 1'

# Add these new samples to the original dataset
X_simulated = np.vstack([X, new_samples])
y_simulated = np.concatenate([y, new_labels])

# Check shapes of the new dataset
print(f"Shape of X_simulated: {X_simulated.shape}")
print(f"Shape of y_simulated: {y_simulated.shape}")

# Proceed with scaling and splitting the simulated data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_simulated)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_simulated, test_size=0.3, random_state=42)

# Check if training and test sets are correctly formed
print(f"Training set shape: X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"Test set shape: X_test: {X_test.shape}, y_test: {y_test.shape}")

# Train the logistic regression model
logistic_model = LogisticRegression(max_iter=1000)

try:
    logistic_model.fit(X_train, y_train)
    print("Model fitted successfully.")
except Exception as e:
    print(f"Error during model fitting: {e}")

# Check if the model is fitted before making predictions
try:
    # Make predictions on the test data
    y_pred = logistic_model.predict(X_test)
    
    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred))
    
    # --- Compute McFadden's R-squared ---
    
    # Calculate log-likelihood of the model
    log_likelihood_model = -log_loss(y_test, logistic_model.predict_proba(X_test))
    
    # Calculate log-likelihood of the null model (assuming no relationship between features and target)
    null_model_probs = np.full_like(y_test, y.mean())  # This is the average probability of the majority class
    log_likelihood_null_model = -log_loss(y_test, null_model_probs)
    
    # Compute McFadden's R-squared
    mcfadden_r2 = 1 - (log_likelihood_model / log_likelihood_null_model)
    print(f"McFadden's R-squared: {mcfadden_r2}")
    
except NotFittedError as e:
    print(f"Model is not fitted: {e}")
except Exception as e:
    print(f"Error during predictions or McFadden calculation: {e}")


Shape of X_simulated: (4008, 1964)
Shape of y_simulated: (4008,)
Training set shape: X_train: (2805, 1964), y_train: (2805,)
Test set shape: X_test: (1203, 1964), y_test: (1203,)
Model fitted successfully.
Accuracy: 0.9800498753117207
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      1179
         1.0       0.00      0.00      0.00        24

    accuracy                           0.98      1203
   macro avg       0.49      0.50      0.49      1203
weighted avg       0.96      0.98      0.97      1203

McFadden's R-squared: 0.7111840885660219


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [112]:
# Set response/dependent variable name
response = 'Remission'

# Create a list of coefficients for features and the intercept
coefficients = list(logistic_model.coef_[0])  # logistic_model is your trained logistic regression model
features = X.columns.tolist()  # List of feature names

# Add the intercept to the features and coefficients lists
features.insert(0, 'Intercept')
coefficients.insert(0, logistic_model.intercept_[0])

# Create a DataFrame with response, feature names, and coefficients
coefficients_df = pd.DataFrame({
    'Response': response,
    'Feature_Name': features,
    'Coef': coefficients
})

# Display the coefficients DataFrame to verify
print(coefficients_df)
# Save the coefficients DataFrame to a CSV file
coeff_filename = './checkpoint/hap786_logistic_regression_coefficients_with_features.csv'
coefficients_df.to_csv(coeff_filename, index=False)

print(f"Coefficients saved to {coeff_filename}")

       Response         Feature_Name       Coef
0     Remission            Intercept -11.206153
1     Remission            age 13-19   0.022174
2     Remission            age 20-40  -0.025311
3     Remission            age 41-64  -0.026491
4     Remission            age 65-79   0.005521
...         ...                  ...        ...
1960  Remission  num_antidepressants  -0.003073
1961  Remission               epi_2p  -0.000624
1962  Remission               rem_2p  -0.008003
1963  Remission              adep_23  -0.000197
1964  Remission              adep_4p  -0.031431

[1965 rows x 3 columns]
Coefficients saved to ./checkpoint/hap786_logistic_regression_coefficients_with_features.csv
